Selenium
---
<a class="anchor" id="selenium"></a>

Often, you're going to need to go to many different websites to scrape data. The package Selenium automates this process for you. 

First, we'll need to download it. 

On a Mac, try typing into your terminal: 

conda install -c conda-forge selenium

Then type:

which chromedriver

If it works, great, if not (or if you have a PC), download it directly from 

http://chromedriver.storage.googleapis.com/index.html 

After you've downloaded it, type "which chromedriver" into your terminal to make sure it gives you a path. Everyone will have a slighty different path. Mine is "/usr/local/bin/chromedriver." Replace what mine is to what yours is in the code below. 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# type `which chromedriver` from shell to find chromedriver. Then change the next line!!!
chromedriver = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(chromedriver)
driver.get("https://images.google.com")

What happened? You should have gotten a browser window that popped up and went to the google images site. Whoa! What if we want to search for Kanye West pics? We'll need to type something into the search text box. Once again, to find how to reference the text box, we'll right click on it and go to Inspect. It seems to be referenced by id=lst-ib so let's tell Chromium to look for that:

In [12]:
search_box = driver.find_element_by_id("lst-ib")
print(search_box)

<selenium.webdriver.remote.webelement.WebElement (session="f4ab5ac08cccb88e640bdfde0d488ca7", element="0.6447736553766905-1")>


Okay, it found it. Now let's tell Chromium to type "Kanye West" into the search box:

In [13]:
search_box.send_keys("Kanye West")

Did you see what just happened on your google webpage? Now, let's press enter to complete our search:

In [14]:
search_box.send_keys(Keys.RETURN)

What if we want to download the first image that comes up? Right click on it and press inspect to see that the the name is "hN6qb_8t9e0lcM:". Let's tell Selenium to search for this name:

In [15]:
search = driver.find_element_by_name('hN6qb_8t9e0lcM:')

Let's click on this pic:

In [16]:
search.click()

Now we are on a new page. If we wanted to know the url of the page we are currently on, we could type:

In [18]:
driver.current_url

'https://www.google.com/search?tbm=isch&source=hp&biw=1200&bih=672&ei=tPimWoWRIMPk0gKEsIugCw&q=Kanye+West&oq=Kanye+West&gs_l=img.3..0l10.9153.9228.0.52295.10.2.0.0.0.0.82.138.2.2.0....0...1ac..64.img..8.2.136....0.5WPJdzhJvVg#imgrc=hN6qb_8t9e0lcM:'

Okay. This new image location is a little harder to find using id or class. Instead, we'll use XPath, which is typically the easiest way to search for something on a webpage. To do this, right click on the Kanye image, choose inspect, and see the blue text that got highlighted. Now, right click on the blue text and choose "Copy - XPath". It should look like this:

<img src="xpath.jpg" style="width: 200px;"/>

Now, we can paste what we just copied into this new search:

In [19]:
pic = search.find_element_by_xpath('//*[@id="irc_cc"]/div[2]/div[1]/div[2]/div[2]/a/img')
pic


<selenium.webdriver.remote.webelement.WebElement (session="f4ab5ac08cccb88e640bdfde0d488ca7", element="0.6236586095908558-2")>

We can use the .get_attribute method to access this hyperlink:

In [20]:
url = pic.get_attribute('src')
print(url)

https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/Kanye_West_at_the_2009_Tribeca_Film_Festival.jpg/1200px-Kanye_West_at_the_2009_Tribeca_Film_Festival.jpg


How can we download this picture? We can use requests to communicate with this url and write its contents to a file called kanye.jpg:

In [21]:
import requests

response = requests.get(url)
with open('kanye.jpg', 'wb') as f:
        f.write(response.content)

### Exercise - Selenium 1

Use Selenium to go to "http://www.boxofficemojo.com/movies/?id=matrix.htm" http://www.boxofficemojo.com and use Selenium to print the Domestic Total Gross.


In [1]:
#insert 1

### Exercise - Selenium 2

Use Selenium to go to http://www.imdb.com/ and to type into the search box Kanye West. You won't quite be there yet because two names are listed. Use Selenium again to click on Kanye West.

In [ ]:
#insert 2

### Selenium and BeautifulSoup

Here's another example. Let's use Selenium to get the temperature for all the zip codes in Portland. Let's first use BeautifulSoup to get the zip codes. If we right click - inspect the zip codes, we see that they are listed under class = Link_List_Text:

In [22]:
import requests
from bs4 import BeautifulSoup
url = 'http://zipcode.org/city/OR/PORTLAND'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

zip_codes = []
data = soup.findAll(class_='List_Link_Text')
for link in data:
    text = link.text
    if 'Zip' in text:
        zip_codes.append(' '.join(text.split()[0:1]))

Next, let's use Selenium to search the temperature for the zip code 97201. Let's use right click - inspect - XPath to find the search box location:

In [59]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(chromedriver)
url = "https://www.weather.gov/"

driver.get(url)
search = driver.find_element_by_xpath('//*[@id="inputstring"]')

'https://forecast.weather.gov/MapClick.php?lat=45.50999000000013&lon=-122.68298999999996#.Wqc0OpPwau4'

Note carefully what you have to do on https://forecast.weather.gov/ to access the weather. You need to click into the text box, type the zip code, press the tab button, and press enter. Let's do that using Selenium:

In [ ]:
search.click()
search.send_keys("97201")
search.send_keys("\t")
search.send_keys(Keys.RETURN)


Unfortunately, we see it didn't work because the website was a little less responsive than our program. Let's add in a few one second delays to give the website time:

In [61]:
import time

chromedriver = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(chromedriver)
url = "https://www.weather.gov/"

driver.get(url)
search = driver.find_element_by_xpath('//*[@id="inputstring"]')

search.click()
search.send_keys("97201")
search.send_keys("\t")
time.sleep(1)
search.send_keys(Keys.RETURN)
time.sleep(1)

Now that we're in the correct spot, we use Selenium to find the temperature by typing right click - inspect - right click - copy XPath. 

Note: Notice at the end of the code below we have a [0].text. Find elements by xpath gave the items back in a list (even though there was only one temperature in it). Therefore, I needed a [0] to reference the first thing in the list.

In [68]:
driver.find_elements_by_xpath('//*[@id="current_conditions-summary"]/p[2]')[0].text

'63°F'

Finally, we can iterate through all of the zip codes we found on the first website to find the temperatures at each and store them to a list.

In [74]:
temps = []

for zip_code in zip_codes:
    
    chromedriver = "/usr/local/bin/chromedriver"
    driver = webdriver.Chrome(chromedriver)
    url = "https://www.weather.gov/"

    driver.get(url)
    search = driver.find_element_by_xpath('//*[@id="inputstring"]')

    search.click()
    search.send_keys(zip_code)
    search.send_keys("\t")
    time.sleep(1)
    search.send_keys(Keys.RETURN)
    time.sleep(1)
    
    temp = driver.find_elements_by_xpath('//*[@id="current_conditions-summary"]/p[2]')[0].text
    print(zip_code, temp)
    temps.append(temp)
    
    driver.close()

97201 63°F
97202 63°F
97203 64°F
97204 63°F
97205 63°F
97206 63°F
97207 63°F
97208 63°F
97209 63°F
97210 63°F
97211 63°F
97212 63°F
97213 63°F
97214 63°F
97220 63°F
97221 63°F
97215 63°F
97216 63°F
97217 63°F
97218 63°F
97219 64°F
97222 63°F
97223 63°F
97227 63°F
97228 63°F
97229 63°F
97231 63°F
97232 63°F
97233 63°F
97236 63°F
97256 66°F
97258 63°F
97266 63°F
97268 66°F
97269 63°F
97280 63°F
97290 63°F
97291 63°F
97292 63°F
97293 63°F
97224 66°F
97225 63°F
97230 63°F
97238 63°F
97239 63°F
97240 63°F
97242 63°F
97267 66°F
97281 66°F
97282 63°F
97283 64°F
97286 63°F
97294 63°F
97296 63°F
97298 63°F


### Exercise - More Selenium 1

Use Selenium to go to "http://www.boxofficemojo.com/movies/?id=matrix.htm" http://www.boxofficemojo.com and use Selenium to print the genres.
Hint: driver.find_elements_by_xpath may contain more than one thing so you might need to loop through it.

In [2]:
#insert 1

### Exercise - More Selenium 2
Use Selenium and XPath to click on the tab that says "Similar Movies." Print the new url location.

In [3]:
#insert 2

### Exercise - More Selenium 3
Use pd.read_html to read in the table of similar movies that you are viewing on the similar movie page that you are now on.

In [4]:
#insert 3

### Exercise - More Selenium 4
Use Selenium to log in to your gmail and send an email to your friend with the subject line "I'M A BOT!"


In [75]:
#insert more selenium 4